In [1]:
#import library
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [2]:
#laod dataset yang akan digunakan untuk test
data = pd.read_csv('data/mental_health.csv')

In [3]:
data

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0
3,im done trying feel betterthe reason im still ...,1
4,worried year old girl subject domestic physic...,1
...,...,...
27972,posting everyday people stop caring religion ...,0
27973,okay definetly need hear guys opinion ive pret...,0
27974,cant get dog think ill kill myselfthe last thi...,1
27975,whats point princess bridei really think like ...,1


In [4]:
data['text']

0        dear american teens question dutch person hear...
1        nothing look forward lifei dont many reasons k...
2        music recommendations im looking expand playli...
3        im done trying feel betterthe reason im still ...
4        worried  year old girl subject domestic physic...
                               ...                        
27972    posting everyday people stop caring  religion ...
27973    okay definetly need hear guys opinion ive pret...
27974    cant get dog think ill kill myselfthe last thi...
27975    whats point princess bridei really think like ...
27976    got nudes person might might know snapchat do ...
Name: text, Length: 27977, dtype: object

In [5]:
text_test = data['text'][1]

text_test

'nothing look forward lifei dont many reasons keep going feel like nothing keeps going next day makes want hang myself'

In [7]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(text_test)
    
endpoint = "http://localhost:8080/v1/models/mental-prediction-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Perlu penanganan pada kesehatan mental" if prediction_value > 0.6 else "Kesehatan mental baik"
else:
    result = "Error: No predictions found in response."

print(result)

Perlu penanganan pada kesehatan mental
